# Validation Type 1

VALIDATION CLASS 0: collect the fake images already generated

In [1]:
import pandas as pd

df = pd.read_csv('C:/Users/nello/Documents/vscode_projects/Thesis/3_image_generation/generated_images/csv/validation_synthetics.csv')

# Filter rows where '6_way_label' equals 0 and 'fake_id' starts with 'SD', 'DL', or 'GL'
filtered_rows_SD = df[(df['6_way_label'] == 0) & (df['fake_id'].str.startswith('SD'))]
filtered_rows_DL = df[(df['6_way_label'] == 0) & (df['fake_id'].str.startswith('DL'))]
filtered_rows_GL = df[(df['6_way_label'] == 0) & (df['fake_id'].str.startswith('GL'))]

# Concatenate columns for all groups
result = pd.concat([filtered_rows_SD, filtered_rows_DL, filtered_rows_GL])
print(len(result))

result.to_csv('C:/Users/nello/Documents/vscode_projects/Thesis/5_biased_detection/csv/validation/validation_class_0_already_generated.csv', index=False)

5602


save a csv for the 11.490 pristine and the 11.490 - 5.602 to be generated

In [25]:
import pandas as pd

MULTIMODAL_VAL_CLEANED_WITH_CLASS_TSV = "C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/val_tsv_with_class.tsv"
ALREADY_GENERATED_FROM_CLASS_0 = "csv/validation/validation_class_0_already_generated.csv"

validation = pd.read_csv(MULTIMODAL_VAL_CLEANED_WITH_CLASS_TSV, sep='\t')
validation_filtered = validation[validation['6_way_label'] == 0]

validation_class_0_already_generated = pd.read_csv(ALREADY_GENERATED_FROM_CLASS_0)
print("len of validation_class_0_already_generated: "+str(len(validation_class_0_already_generated)))

# Filter rows from train DataFrame that are not in the 'id' column of the other DataFrame
validation_not_in_other = validation_filtered[~validation_filtered['id'].isin(validation_class_0_already_generated['id'])]

# Select the first 11,490 rows from train_not_in_other
validation_pristine = validation_not_in_other.head(11491)

# Save train_pristine to CSV
print("len of validation_class_0_pristine: "+str(len(validation_pristine)))
validation_pristine.to_csv('csv/validation/validation_class_0_pristine.csv', index=False)

# Filter rows from train_filtered that are not in train_class_0_already_generated or train_pristine
validation_remaining = validation_filtered[~validation_filtered['id'].isin(validation_class_0_already_generated['id']) & ~validation_filtered['id'].isin(validation_pristine['id'])]

# Save the remaining rows to a new CSV file
print("len of validation_class_0_to_be_generated: "+str(len(validation_remaining)))
validation_remaining.to_csv('csv/validation/validation_class_0_to_be_generated.csv', index=False)
print("validation_class_0_already_generated: "+str(len(validation_class_0_already_generated))+" + validation_class_0_to_be_generated: "+str(len(validation_remaining))+" = " + str(len(validation_class_0_already_generated)+len(validation_remaining)))

len of validation_class_0_already_generated: 5602
len of validation_class_0_pristine: 11491
len of validation_class_0_to_be_generated: 5888
validation_class_0_already_generated: 5602 + validation_class_0_to_be_generated: 5888 = 11490


rename the columns real_image and fake_image to pristine_image and generated_image inside validation_class_0_already_generated.csv

In [26]:
df = pd.read_csv('csv/validation/validation_class_0_already_generated.csv')

# Rename the column
df.rename(columns={"real_image": "pristine_image"}, inplace=True)
df.rename(columns={"fake_image": "generated_image"}, inplace=True)
df.to_csv("csv/validation/validation_class_0_already_generated.csv", index=False)

also change the value of columns real_image and fake_image in validation_class_0_already_generated.csv from real_image=1 and fake_image=0 to real_image=0 and fake_image=1

In [33]:
df = pd.read_csv('csv/validation/validation_class_0_already_generated.csv')

# Swap the values of the two columns
df['pristine_image'], df['generated_image'] = df['generated_image'], df['pristine_image']

df.to_csv("csv/validation/validation_class_0_already_generated.csv", index=False)

reduce columns of validation_class_0_to_be_generated and validation_class_0_pristine csv:

In [34]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_class_0_to_be_generated.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 1
df['fakenews_text'] = 0

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/validation/validation_class_0_to_be_generated.csv", index=False)

In [35]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_class_0_pristine.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 1
df['fakenews_text'] = 0

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/validation/validation_class_0_pristine.csv", index=False)

check if every id inside the validation_tsv_with_class.csv is contained in the union of the 3 csv validation_pristine + validation_class_0_already_generated + validation_class_0_to_be_generated

In [30]:
import pandas as pd

# Load the 'id' column from the first three CSV files into sets
csv1_ids = set(pd.read_csv("csv/validation/validation_class_0_pristine.csv")['id'])
csv2_ids = set(pd.read_csv("csv/validation/validation_class_0_to_be_generated.csv")['id'])
csv3_ids = set(pd.read_csv("csv/validation/validation_class_0_already_generated.csv")['id'])

# Find the union of these three sets
union_ids = csv1_ids.union(csv2_ids, csv3_ids)

csv4_ids = set(pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/val_tsv_with_class.tsv", sep='\t').loc[pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/val_tsv_with_class.tsv", sep='\t')['6_way_label'] == 0, 'id'])

print(len(union_ids))
print(len(csv4_ids))

# Check if the union_ids set is equal to the csv4_ids set
if union_ids == csv4_ids:
    print("The union of IDs from the first three CSVs corresponds exactly to the IDs from validation_tsv_with_class.csv where '6_way_label' = 0.")
else:
    print("The union of IDs from the first three CSVs does not correspond exactly to the IDs from validation_tsv_with_class.csv where '6_way_label' = 0.")

22981
22981
The union of IDs from the first three CSVs corresponds exactly to the IDs from validation_tsv_with_class.csv where '6_way_label' = 0.


Now i print out how many StableDiffusion, Dreamlike and Glide images i have already generated:

In [31]:
df = pd.read_csv("csv/validation/validation_class_0_already_generated.csv")

# Count the rows where the 'fake_id' column starts with 'SD', 'DL', or 'GL'
count_SD = len(df[df['fake_id'].str.startswith('SD')])
count_DL = len(df[df['fake_id'].str.startswith('DL')])
count_GL = len(df[df['fake_id'].str.startswith('GL')])

print("Number of rows where 'fake_id' starts with 'SD':", count_SD)
print("Number of rows where 'fake_id' starts with 'DL':", count_DL)
print("Number of rows where 'fake_id' starts with 'GL':", count_GL)

Number of rows where 'fake_id' starts with 'SD': 2263
Number of rows where 'fake_id' starts with 'DL': 2242
Number of rows where 'fake_id' starts with 'GL': 1097


Thus, to get to 11.490 generated images i want to generate other:

- SD: 4.000 - 2.263 = 1.737
- DL: 4.000 - 2.242 = 1.758
- GL: (11.490 - 8.000) - 1.097 = 2.393

so 5.888 in total from the validation_class_0_to_be_generated.csv, but first i need to caption all the images from validation_class_0_to_be_generated.csv

Here we check that every pristine image inside validation_class_0_to_be_generated csv is already in the image folder

In [36]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/validation/validation_class_0_to_be_generated.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 5888


Here we check that every pristine image inside validation_class_0_pristine csv is already in the image folder

In [37]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/validation/validation_class_0_pristine.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 11491


Here we check that every pristine image inside validation_class_0_already_generated csv is already in the image folder

In [38]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/validation/validation_class_0_already_generated.csv")['fake_id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 5602


Clean the validation_class_0_captioned csv, deleting the generated captions that starts with "araf" (buggy captions)

In [ ]:
import pandas as pd

original_df = pd.read_csv('/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/validation/validation_class_0_captioned.csv')

# Create a copy of the original DataFrame
modified_df = original_df.copy()

# Remove words starting with "araf" in the "generated_caption" column
modified_df['generated_caption'] = modified_df['generated_caption'].str.replace(r'\baraf\w*\b', '', regex=True)

# Remove leading and trailing whitespace from the "generated_caption" column
modified_df['generated_caption'] = modified_df['generated_caption'].str.strip()

# Save the modified DataFrame to the same CSV file
modified_df.to_csv("/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/validation/validation_class_0_captioned.csv", index=False)

# Count the number of updated rows
updated_rows_count = (original_df['generated_caption'] != modified_df['generated_caption']).sum()

print(f"Number of updated rows: {updated_rows_count}")

-------------------------

# Validation Type 2

Now construct the validation "type 2" part, thus the 8.963 pristine with 6_way_label == 1,2,3,5 (4 manipulated deleted) and 8.963 generated still with 6_way_label == 1,2,3,5

In [7]:
import pandas as pd

df = pd.read_csv('C:/Users/nello/Documents/vscode_projects/Thesis/3_image_generation/generated_images/csv/validation_synthetics.csv')

# Filter rows where '6_way_label' equals 1, 2, 3, or 5 and 'fake_id' starts with 'SD', 'DL', or 'GL'
filtered_rows = df[(df['6_way_label'].isin([1, 2, 3, 5])) & (df['fake_id'].str.startswith(('SD', 'DL', 'GL')))]

# Save the result to a new CSV file
print(len(filtered_rows))
filtered_rows.to_csv('csv/validation/validation_class_1_2_3_5_already_generated.csv', index=False)

4398


save a csv for the 8.963 pristine and the 8.963-4.398 to be generated

In [39]:
import pandas as pd

MULTIMODAL_VAL_CLEANED_WITH_CLASS_TSV = "C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/val_tsv_with_class.tsv"
ALREADY_GENERATED_FROM_CLASS_1_2_3_5 = "csv/validation/validation_class_1_2_3_5_already_generated.csv"

validation = pd.read_csv(MULTIMODAL_VAL_CLEANED_WITH_CLASS_TSV, sep='\t')
print("validation len: "+str(len(validation)))
validation_filtered = validation[validation['6_way_label'].isin([1, 2, 3, 5])]

validation_class_1_2_3_5_already_generated = pd.read_csv(ALREADY_GENERATED_FROM_CLASS_1_2_3_5)
print("len of validation_class_1_2_3_5_already_generated: "+str(len(validation_class_1_2_3_5_already_generated)))

# Filter rows from val DataFrame that are not in the 'id' column of the other DataFrame
validation_not_in_other = validation_filtered[~validation_filtered['id'].isin(validation_class_1_2_3_5_already_generated['id'])]

# Select the first 8.963 rows from validation_not_in_other
validation_pristine = validation_not_in_other.head(8964)

# Save validation_pristine to CSV
print("len of validation_class_1_2_3_5_pristine: "+str(len(validation_pristine)))
validation_pristine.to_csv('csv/validation/validation_class_1_2_3_5_pristine.csv', index=False)

# Filter rows from validation_filtered that are not in validation_class_1_2_3_5_already_generated or validation_pristine
validation_remaining = validation_filtered[~validation_filtered['id'].isin(validation_class_1_2_3_5_already_generated['id']) & ~validation_filtered['id'].isin(validation_pristine['id'])]

# Save the remaining rows to a new CSV file
print("len of validation_class_1_2_3_5_to_be_generated: "+str(len(validation_remaining)))
validation_remaining.to_csv('csv/validation/validation_class_1_2_3_5_to_be_generated.csv', index=False)
print("validation_class_1_2_3_5_already_generated: "+str(len(validation_class_1_2_3_5_already_generated))+"  + validation_class_1_2_3_5_to_be_generated: "+str(len(validation_remaining))+" = " + str(len(validation_class_1_2_3_5_already_generated)+len(validation_remaining)))

validation len: 57489
len of validation_class_1_2_3_5_already_generated: 4398
len of validation_class_1_2_3_5_pristine: 8964
len of validation_class_1_2_3_5_to_be_generated: 4566
validation_class_1_2_3_5_already_generated: 4398  + validation_class_1_2_3_5_to_be_generated: 4566 = 8964


rename the columns real_image and fake_image to pristine_image and generated_image inside validation_class_1_2_3_5_already_generated.csv

In [40]:
df = pd.read_csv('csv/validation/validation_class_1_2_3_5_already_generated.csv')

# Rename the column
df.rename(columns={"real_image": "pristine_image"}, inplace=True)
df.rename(columns={"fake_image": "generated_image"}, inplace=True)
df.to_csv("csv/validation/validation_class_1_2_3_5_already_generated.csv", index=False)

also change the value of columns real_image and fake_image in validation_class_1_2_3_5_already_generated.csv from real_image=1 and fake_image=0 to real_image=0 and fake_image=1

In [41]:
df = pd.read_csv('csv/validation/validation_class_1_2_3_5_already_generated.csv')

# Swap the values of the two columns
df['pristine_image'], df['generated_image'] = df['generated_image'], df['pristine_image']

df.to_csv("csv/validation/validation_class_1_2_3_5_already_generated.csv", index=False)

reduce columns of validation_class_1_2_3_5_to_be_generated and validation_class_1_2_3_5_pristine csv:

In [42]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_class_1_2_3_5_to_be_generated.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 0
df['fakenews_text'] = 1

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/validation/validation_class_1_2_3_5_to_be_generated.csv", index=False)

In [43]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_class_1_2_3_5_pristine.csv')

# Rename the column
df.rename(columns={"clean_title": "original_caption"}, inplace=True)

df['pristine_image'] = 1
df['generated_image'] = 0
df['real_text'] = 0
df['fakenews_text'] = 1

# Select only the desired columns in the required order
desired_columns = ['id', 'author', 'original_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']
df = df[desired_columns]

df.to_csv("csv/validation/validation_class_1_2_3_5_pristine.csv", index=False)

Show the distribution of the original classes (1,2,3,5) for the csv

In [44]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_class_1_2_3_5_pristine.csv')

# Count the number of rows for each value of '6_way_label'
label_counts = df['6_way_label'].value_counts()

# Print the counts for each unique value of '6_way_label'
print("Counts for each unique value of '6_way_label':")
print(label_counts)

Counts for each unique value of '6_way_label':
6_way_label
2    5570
1    1701
5    1082
3     611
Name: count, dtype: int64


In [45]:
df = pd.read_csv('csv/validation/validation_class_1_2_3_5_already_generated.csv')

# Count the number of rows for each value of '6_way_label'
label_counts = df['6_way_label'].value_counts()

# Print the counts for each unique value of '6_way_label'
print("Counts for each unique value of '6_way_label':")
print(label_counts)

Counts for each unique value of '6_way_label':
6_way_label
2    2696
1     890
5     520
3     292
Name: count, dtype: int64


In [46]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_class_1_2_3_5_already_generated.csv')

# Group the DataFrame by '6_way_label'
grouped = df.groupby('6_way_label')

# Iterate over each group
for label, group in grouped:
    # Filter the group where 'id' column starts with "SD", "GL", or "DL"
    sd_count = len(group[group['fake_id'].str.startswith('SD')])
    gl_count = len(group[group['fake_id'].str.startswith('GL')])
    dl_count = len(group[group['fake_id'].str.startswith('DL')])
    
    # Print counts for each value of '6_way_label'
    print(f"Counts for '6_way_label' {label}:")
    print(f"Starts with 'SD': {sd_count}")
    print(f"Starts with 'GL': {gl_count}")
    print(f"Starts with 'DL': {dl_count}")
    print()


Counts for '6_way_label' 1:
Starts with 'SD': 348
Starts with 'GL': 186
Starts with 'DL': 356

Counts for '6_way_label' 2:
Starts with 'SD': 1065
Starts with 'GL': 552
Starts with 'DL': 1079

Counts for '6_way_label' 3:
Starts with 'SD': 119
Starts with 'GL': 50
Starts with 'DL': 123

Counts for '6_way_label' 5:
Starts with 'SD': 205
Starts with 'GL': 115
Starts with 'DL': 200



In [2]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_class_1_2_3_5_already_generated.csv')

# Calculate the count of rows starting with 'SD', 'DL', and 'GL'
sd_count = len(df[df['fake_id'].str.startswith('SD')])
dl_count = len(df[df['fake_id'].str.startswith('DL')])
gl_count = len(df[df['fake_id'].str.startswith('GL')])

# Print counts for each case
print(f"Count of rows starting with 'SD': {sd_count}")
print(f"Count of rows starting with 'DL': {dl_count}")
print(f"Count of rows starting with 'GL': {gl_count}")

Count of rows starting with 'SD': 1737
Count of rows starting with 'DL': 1758
Count of rows starting with 'GL': 903


Thus, to get to 8.963 generated images i want to generate other 8.963-4398 = 4.565:

- SD: 3.000 - 1.737 = 1.263
- DL: 3.000 - 1.758 = 1.242
- GL: (8.963 - 6.000) - 903 = 2.060

(3.000 means i want 3.000 images fakes from SD out of 8.963 total fakes)

so 4.565 in total from the validation_class_1_2_3_5_to_be_generated.csv, but first i need to caption all the images from validation_class_1_2_3_5_to_be_generated.csv

In [3]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('csv/validation/validation_class_1_2_3_5_to_be_generated.csv')

# Define the start and end indices for each interval
intervals = {
    "Generated with SD": (0, 1263),
    "Generated with DL": (1263, 2505),
    "Generated with GL": (2505, len(df))
}

# Iterate over each interval
for method, (start_index, end_index) in intervals.items():
    # Filter the DataFrame based on the interval
    interval_df = df.iloc[start_index:end_index]
    
    # Group the interval DataFrame by '6_way_label'
    grouped = interval_df.groupby('6_way_label')
    
    # Print the count of rows with each '6_way_label' value within the interval
    print(f"{method}:")
    for label, group in grouped:
        label_count = len(group)
        print(f"6_way_label: {label} in the interval [{start_index}, {end_index}): {label_count}")
    print()

Generated with SD:
6_way_label: 1 in the interval [0, 1263): 247
6_way_label: 2 in the interval [0, 1263): 769
6_way_label: 3 in the interval [0, 1263): 87
6_way_label: 5 in the interval [0, 1263): 160

Generated with DL:
6_way_label: 1 in the interval [1263, 2505): 225
6_way_label: 2 in the interval [1263, 2505): 782
6_way_label: 3 in the interval [1263, 2505): 76
6_way_label: 5 in the interval [1263, 2505): 159

Generated with GL:
6_way_label: 1 in the interval [2505, 4566): 375
6_way_label: 2 in the interval [2505, 4566): 1299
6_way_label: 3 in the interval [2505, 4566): 143
6_way_label: 5 in the interval [2505, 4566): 244



check if every id inside the validation_tsv_with_class.csv with '6_way_label' = 1,2,3,5 is contained in the union of the 3 csv validation_class_1_2_3_5_pristine + validation_class_1_2_3_5_already_generated + validation_class_1_2_3_5_to_be_generated

In [12]:
import pandas as pd

# Load the 'id' column from the first three CSV files into sets
csv1_ids = set(pd.read_csv("csv/validation/validation_class_1_2_3_5_pristine.csv")['id'])
csv2_ids = set(pd.read_csv("csv/validation/validation_class_1_2_3_5_to_be_generated.csv")['id'])
csv3_ids = set(pd.read_csv("csv/validation/validation_class_1_2_3_5_already_generated.csv")['id'])

# Find the union of these three sets
union_ids = csv1_ids.union(csv2_ids, csv3_ids)

csv4_ids = set(pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/val_tsv_with_class.tsv", sep='\t').loc[pd.read_csv("C:/Users/nello/OneDrive - University of Pisa/TESI/TSV_JSON/1_dataset_cleaning/tsv/train_tsv_with_class.tsv", sep='\t')['6_way_label'].isin([1, 2, 3, 5]), 'id'])

print(len(union_ids))
print(len(csv4_ids))

# Check if the union_ids set is equal to the csv4_ids set
if union_ids == csv4_ids:
    print("The union of IDs from the first three CSVs corresponds exactly to the IDs from validation_tsv_with_class.csv where '6_way_label' = 1,2,3,5.")
else:
    print("The union of IDs from the first three CSVs does not correspond exactly to the IDs from validation_tsv_with_class.csv where '6_way_label' = 1,2,3,5.")

170774
170774
The union of IDs from the first three CSVs corresponds exactly to the IDs from train_tsv_with_class.csv where '6_way_label' = 1,2,3,5.


Here we check that every pristine image inside validation_class_1_2_3_5_to_be_generated csv is already in the image folder

In [3]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/validation/validation_class_1_2_3_5_to_be_generated.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 4566


Here we check that every pristine image inside validation_class_1_2_3_5_pristine csv is already in the image folder

In [4]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/validation/validation_class_1_2_3_5_pristine.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 8964


Here we check that every pristine image inside validation_class_1_2_3_5_already_generated csv is already in the image folder

In [5]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/validation/validation_class_1_2_3_5_already_generated.csv")['fake_id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 4398


Clean the validation_class_1_2_3_5_captioned csv, deleting the generated captions that starts with "araf" (buggy captions)

In [2]:
import pandas as pd

original_df = pd.read_csv('/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/validation/validation_class_1_2_3_5_captioned.csv')

# Create a copy of the original DataFrame
modified_df = original_df.copy()

# Remove words starting with "araf" in the "generated_caption" column
modified_df['generated_caption'] = modified_df['generated_caption'].str.replace(r'\baraf\w*\b', '', regex=True)

# Remove leading and trailing whitespace from the "generated_caption" column
modified_df['generated_caption'] = modified_df['generated_caption'].str.strip()

# Save the modified DataFrame to the same CSV file
modified_df.to_csv("/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/validation/validation_class_1_2_3_5_captioned.csv", index=False)

# Count the number of updated rows
updated_rows_count = (original_df['generated_caption'] != modified_df['generated_caption']).sum()

print(f"Number of updated rows: {updated_rows_count}")

Number of updated rows: 1528


------------------

# After fake generation

After generating all the fakes images that were missing, i need to merge the class_0_to_be_generated and the class_0_already_generated, and the same for class 1,2,3,5

Starting by adding the "fake_id" column to the validation_class_0_to_be_generated csv

In [1]:
import pandas as pd

df = pd.read_csv("/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/validation/validation_class_0_to_be_generated.csv")

# new column 'fake_id'
df['fake_id'] = ''

# assign values to 'fake_id' based on row index intervals
df.loc[df.index < 1737, 'fake_id'] = 'SD_fake_' + df['id']
df.loc[(df.index >= 1737) & (df.index < 3495), 'fake_id'] = 'DL_fake_' + df['id']
df.loc[df.index >= 3495, 'fake_id'] = 'GL_fake_' + df['id']

# 'fake_id' column next to 'id' column
df.insert(df.columns.get_loc('id') + 1, 'fake_id', df.pop('fake_id'))

df.to_csv('/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/validation/validation_class_0_final.csv', index=False)

In [2]:
import pandas as pd

df = pd.read_csv("/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/validation/validation_class_1_2_3_5_to_be_generated.csv")

# new column 'fake_id'
df['fake_id'] = ''

# assign values to 'fake_id' based on row index intervals
df.loc[df.index < 1263, 'fake_id'] = 'SD_fake_' + df['id']
df.loc[(df.index >= 1263) & (df.index < 2505), 'fake_id'] = 'DL_fake_' + df['id']
df.loc[df.index >= 2505, 'fake_id'] = 'GL_fake_' + df['id']

# 'fake_id' column next to 'id' column
df.insert(df.columns.get_loc('id') + 1, 'fake_id', df.pop('fake_id'))

df.to_csv('/home/enriconello/DeepFakeDetection/Thesis/5_biased_detection/csv/validation/validation_class_1_2_3_5_final.csv', index=False)

set the column pristine_image = 0 and generated_image = 1 for every row in validation_class_0_final csv

In [3]:
df = pd.read_csv('csv/validation/validation_class_0_final.csv')

# Swap the values of the two columns
df['pristine_image'], df['generated_image'] = df['generated_image'], df['pristine_image']

df.to_csv("csv/validation/validation_class_0_final.csv", index=False)

In [4]:
df = pd.read_csv('csv/validation/validation_class_1_2_3_5_final.csv')

# Swap the values of the two columns
df['pristine_image'], df['generated_image'] = df['generated_image'], df['pristine_image']

df.to_csv("csv/validation/validation_class_1_2_3_5_final.csv", index=False)

Now i need to add the column "generated_caption"

In [5]:
import pandas as pd

df_a = pd.read_csv("csv/validation/validation_class_0_final.csv")
df_b = pd.read_csv("csv/validation/validation_class_0_captioned.csv")

# Perform inner join on 'id' column
df_c = pd.merge(df_a, df_b, on='id', how='inner')

# Rename 'original_caption_x' to 'original_caption'
df_c = df_c.rename(columns={'original_caption_x': 'original_caption'})

df_c = df_c[['id', 'fake_id', 'author', 'original_caption', 'generated_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']]

df_c.to_csv("csv/validation/validation_class_0_final.csv", index=False)
df_c.head()

,id,fake_id,author,original_caption,generated_caption,num_comments,6_way_label,pristine_image,generated_image,real_text,fakenews_text
0,blkphy,SD_fake_blkphy,jespeland01,sitting on the edge of a cliff,a man sitting on a rock looking out over a valley,1.0,0,0,1,1,0
1,c0e0xx,SD_fake_c0e0xx,Zirocket,this building beside the train station painted...,there are two trains that are on the tracks in...,0.0,0,0,1,1,0
2,cddj2g,SD_fake_cddj2g,courtysim,orourke says he and wife descended from slave ...,man in a white shirt and tie holding a microphone,7.0,0,0,1,1,0
3,c9kpo8,SD_fake_c9kpo8,andrewgardnr,this is a phone screen under a microscope,a close up of a light that is shining brightly...,6.0,0,0,1,1,0
4,7u9y4x,SD_fake_7u9y4x,sisyphushaditsoeasy,students parents surprise school bus driver wi...,man in a green hat is hugging a woman,0.0,0,0,1,1,0


In [7]:
import pandas as pd

df_a = pd.read_csv("csv/validation/validation_class_1_2_3_5_final.csv")
df_b = pd.read_csv("csv/validation/validation_class_1_2_3_5_captioned.csv")

# Perform inner join on 'id' column
df_c = pd.merge(df_a, df_b, on='id', how='inner')

# Rename 'original_caption_x' to 'original_caption'
df_c = df_c.rename(columns={'original_caption_x': 'original_caption'})

df_c = df_c[['id', 'fake_id', 'author', 'original_caption', 'generated_caption', 'num_comments', '6_way_label', 'pristine_image', 'generated_image', 'real_text', 'fakenews_text']]

df_c.to_csv("csv/validation/validation_class_1_2_3_5_final.csv", index=False)
df_c.head()

,id,fake_id,author,original_caption,generated_caption,num_comments,6_way_label,pristine_image,generated_image,real_text,fakenews_text
0,9hypej,SD_fake_9hypej,Yessod,i cannot believe this wasnt done on purpose,there are two trash bins that are sitting on t...,0.0,2,0,1,0,1
1,c94gks,SD_fake_c94gks,xyzzyx13,jumping dolphin at moonlight,there is a drawing of a dolphin on a tile wall,3.0,2,0,1,0,1
2,4905w5,SD_fake_4905w5,rexlibris,avec le agissez algerian war of independence s,a red poster with a hand holding a cigarette a...,4.0,5,0,1,0,1
3,78fxbp,SD_fake_78fxbp,YouBecame,this man asked a simple question online that s...,a black and white photo of a man with a hand o...,11.0,5,0,1,0,1
4,9gfob0,SD_fake_9gfob0,RadioOphiuchi,hasselblad how quaint,there is a phone booth with a phone on the out...,0.0,1,0,1,0,1


finally merge validation_class_0_final and validation_class_1_2_3_5_final to create validation_generated_final csv that contains all the generated images

In [14]:
import pandas as pd

df1 = pd.read_csv("csv/validation/validation_class_0_final.csv")
df3 = pd.read_csv("csv/validation/validation_class_0_already_generated.csv")
df2 = pd.read_csv("csv/validation/validation_class_1_2_3_5_final.csv")
df4 = pd.read_csv("csv/validation/validation_class_1_2_3_5_already_generated.csv")

# Concatenate DataFrames
df_union = pd.concat([df1, df2, df3, df4], ignore_index=True)

df_union.to_csv("csv/validation/validation_generated_final.csv", index=False)
print(len(df_union))

20454


Print out the cardinalities of the classes with the generation methods associated

In [15]:
import pandas as pd

df = pd.read_csv('csv/validation/validation_generated_final.csv')

# Group the DataFrame by '6_way_label'
grouped = df.groupby('6_way_label')

# Iterate over each group
for label, group in grouped:
    # Filter the group where 'id' column starts with "SD", "GL", or "DL"
    sd_count = len(group[group['fake_id'].str.startswith('SD')])
    gl_count = len(group[group['fake_id'].str.startswith('GL')])
    dl_count = len(group[group['fake_id'].str.startswith('DL')])
    
    # Print counts for each value of '6_way_label'
    print(f"Counts for '6_way_label' {label}:")
    print(f"Starts with 'SD': {sd_count}")
    print(f"Starts with 'GL': {gl_count}")
    print(f"Starts with 'DL': {dl_count}")
    print()

Counts for '6_way_label' 0:
Starts with 'SD': 4000
Starts with 'GL': 3490
Starts with 'DL': 4000

Counts for '6_way_label' 1:
Starts with 'SD': 595
Starts with 'GL': 561
Starts with 'DL': 581

Counts for '6_way_label' 2:
Starts with 'SD': 1834
Starts with 'GL': 1851
Starts with 'DL': 1861

Counts for '6_way_label' 3:
Starts with 'SD': 206
Starts with 'GL': 193
Starts with 'DL': 199

Counts for '6_way_label' 5:
Starts with 'SD': 365
Starts with 'GL': 359
Starts with 'DL': 359



Now i need to merge validation_class_0_pristine and validation_class_1_2_3_5_pristine to make validation_pristine_final csv

In [10]:
import pandas as pd

df1 = pd.read_csv("csv/validation/validation_class_0_pristine.csv")
df2 = pd.read_csv("csv/validation/validation_class_1_2_3_5_pristine.csv")

# Concatenate DataFrames
df_union = pd.concat([df1, df2], ignore_index=True)

df_union.to_csv("csv/validation/validation_pristine_final.csv", index=False)
print(len(df_union))

20455


Finally i need to merge the validation_pristine_final with validation_generated_final to generated validation csv

In [16]:
import pandas as pd

df_a = pd.read_csv('csv/validation/validation_generated_final.csv')
df_b = pd.read_csv('csv/validation/validation_pristine_final.csv')

# Add missing columns from A to B with empty values
for column in df_a.columns:
    if column not in df_b.columns:
        df_b[column] = ''

# Concatenate dataframes
df_c = pd.concat([df_a, df_b])

print(len(df_c))
df_c.to_csv('csv/validation/validation.csv', index=False)

40909


All the images are already inside the images folder because the validation was "with duplicates", so also the real samples were maintained together with the fake ones. So for validation i should just add to the images folder the new generated images (those inside validation/class_0 and validation/class_1_2_3_5), and also duplicate the rows of the new generated images in the csv for the real samples.

In [24]:
import os
import pandas as pd

# Load the 'id' column from the CSV file into a set
csv_ids = set(pd.read_csv("csv/validation/validation.csv")['id'])

# Get a list of all the image filenames in the folder
folder_path = "/home/enriconello/DeepFakeDetection/dataset"
folder_images = set(filename.split('.')[0] for filename in os.listdir(folder_path))

# Find the intersection to identify matching images
matching_images = csv_ids.intersection(folder_images)

# Count the number of matching images
num_matching_images = len(matching_images)

print("Number of matching images:", num_matching_images)

Number of matching images: 40909
